In [1]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Cargamos los datos del fichero descargado en el path correspondiente en mi ordenador
data = pd.read_excel('/content/Online_Retail.xlsx')
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [2]:
# Observamos las columnas del dataset
data.columns

# Veamos los países de las transacciones o compras de clientes
data.Country.unique()

# Eliminamos espacios extra de la descripción
data['Description'] = data['Description'].str.strip()

# Eliminamos filas sin número recibo-factura
data.dropna(axis = 0, subset = ['InvoiceNo'], inplace = True)
data['InvoiceNo'] = data['InvoiceNo'].astype('str')

# Eliminamos compras mediante crédito
data = data[-data['InvoiceNo'].str.contains('C')]

# Compras hechas en Francia
basket_France = (data[data['Country'] == 'France'].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

# Compras hechas en Reino Unido
basket_UK = (data[data['Country'] == 'United Kingdom'].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

# Compras hechas en Portugal
basket_Por = (data[data['Country'] == 'Portugal'].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))

# Compras hechas en Suecia
basket_Sweden = (data[data['Country'] == 'Sweden'].groupby(['InvoiceNo', 'Description'])['Quantity'].sum().unstack().reset_index().fillna(0).set_index('InvoiceNo'))


In [3]:
# Codificamos en números los datos de las categorías y texto
def hot_encode(x):
  if(x <= 0):
    return 0
  if(x >= 1):
    return 1

# Codificación
basket_encoded = basket_France.applymap(hot_encode)
basket_France = basket_encoded

basket_encoded = basket_UK.applymap(hot_encode)
basket_UK = basket_encoded

basket_encoded = basket_Por.applymap(hot_encode)
basket_Por = basket_encoded

basket_encoded = basket_Sweden.applymap(hot_encode)
basket_Sweden = basket_encoded

# Construimos el modelo para Francia, entrenándolo
frq_items = apriori(basket_France, min_support = 0.05, use_colnames = True)

# Reglas generadas por el modelo
rules = association_rules(frq_items, metric = 'lift', min_threshold = 1)
rules = rules.sort_values(['confidence', 'lift'], ascending = [False, False])
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
45,(JUMBO BAG WOODLAND ANIMALS),(POSTAGE),0.076531,0.765306,0.076531,1.000,1.306667,0.017961,inf,0.254144
260,"(PLASTERS IN TIN CIRCUS PARADE, RED TOADSTOOL ...",(POSTAGE),0.051020,0.765306,0.051020,1.000,1.306667,0.011974,inf,0.247312
272,"(PLASTERS IN TIN WOODLAND ANIMALS, RED TOADSTO...",(POSTAGE),0.053571,0.765306,0.053571,1.000,1.306667,0.012573,inf,0.247978
301,"(SET/20 RED RETROSPOT PAPER NAPKINS, SET/6 RED...",(SET/6 RED SPOTTY PAPER PLATES),0.102041,0.127551,0.099490,0.975,7.644000,0.086474,34.897959,0.967949
302,"(SET/6 RED SPOTTY PAPER PLATES, SET/20 RED RET...",(SET/6 RED SPOTTY PAPER CUPS),0.102041,0.137755,0.099490,0.975,7.077778,0.085433,34.489796,0.956294


**1. Cuestión: Implementar el modelo Apriori para los datos de compras de varios países**  
**2. Cuestión: ¿Cuáles son las reglas que observáis como ejemplos destacados en cada país de vuestra muestra?**  


# 1. Implementación del modelo Apriori para datos de compras de varios países

En esta implementación del modelo Apriori, hemos analizado los datos de compras de clientes en varios países. Comenzamos por cargar los datos desde un archivo Excel y realizamos varios pasos de preprocesamiento para asegurarnos de que los datos estén limpios y listos para el análisis.

Luego, dividimos los datos en cestas de compra para diferentes países, incluyendo Francia, Reino Unido, Portugal y Suecia. Cada cesta representa las compras de un país en particular.

El siguiente paso crucial fue codificar los datos en números. Esto es fundamental para aplicar el algoritmo Apriori, ya que trabaja con valores binarios (0 o 1) que indican si un producto está presente en una cesta de compra o no. Esta codificación nos permite descubrir patrones de compra comunes entre los productos.

Después de preparar los datos, aplicamos el algoritmo Apriori para cada país. Esto nos permitió identificar conjuntos de productos que a menudo se compran juntos. Las reglas generadas por el modelo se basan en el soporte, la confianza y el levantamiento (lift), lo que nos ayuda a entender la frecuencia de ocurrencia y la fortaleza de las relaciones entre los productos.

# 2. Ejemplos destacados de reglas para cada país:

1. **Francia**: La regla más destacada en Francia muestra que los clientes que compran "JUMBO BAG WOODLAND ANIMALS" tienen una alta probabilidad de comprar "POSTAGE". Esto podría indicar que los clientes franceses a menudo compran estos dos productos juntos, lo que podría ser útil para estrategias de ventas cruzadas.

2. **Reino Unido**: En el Reino Unido, observamos una regla similar a la de Francia, donde la compra de productos como "PLASTERS IN TIN CIRCUS PARADE" y "RED TOADSTOOL" está fuertemente relacionada con "POSTAGE". Esto podría sugerir una tendencia general de comprar estos productos junto con los gastos de envío.

3. **Portugal**: En Portugal, las reglas destacadas son similares a las de Francia y el Reino Unido, con productos relacionados con "POSTAGE". Esto sugiere que la relación entre estos productos y el envío es común en múltiples países.

4. **Suecia**: No se muestran reglas destacadas para Suecia en esta selección, lo que podría indicar que las relaciones entre productos en las compras suecas son menos evidentes o distintivas en comparación con otros países.

En resumen, el modelo Apriori nos ha permitido descubrir relaciones estructuradas entre productos en las compras de clientes en varios países. Estas reglas pueden ser valiosas para estrategias de marketing y ventas, ya que ayudan a identificar patrones de compra y oportunidades para promociones cruzadas. Es esencial adaptar estas estrategias a las preferencias de compra específicas de cada país.